# Generalizations of Max-Flow

The purpose of this assignment is to learn about the min-cost flow problem, a generalization of max-flow, and to familiarize yourself with implementing and solving linear programs.

## Min-Cost Flow

Recall that a flow network with demands consists of a directed graph $G = (V, E)$, where each edge $(i,j) \in E$ has a positive integer capacity $c_{ij}$ and each node $i \in V$ has an integer demand $d_i$. In a *min-cost flow* problem, each edge $(i,j) \in E$ also has a cost (or weight) $w_{ij}$. (Note that this input generalizes the input to two important problems we discussed so far: in max flow, the edge weights were not important while in shortest paths, the edge capacities were not important.) 

Given a flow network with capacities and costs, the goal is to find a *feasible* flow $f: E \rightarrow R^+$ --that is, a flow satisfying edge capacity constraints and node demands-- that minimizes the total cost of the flow. Explicitly, the problem can be formulated as a linear program.

### Question 1

Answer Problem 1 in HW4-theoretical.

### Question 2

To implement your reduction from Problem 1 in HW4-theoretical, you will work with some standard benchmark instances for min-cost flow found [here](http://elib.zib.de/pub/Packages/mp-testdata/mincost/gte/index.html). The format of the data is described in the [Info](http://elib.zib.de/pub/Packages/mp-testdata/mincost/gte/info) file. You are to read in the graph from the data file in a form that can be solved using NetworkX's `min_cost_flow` function. Note that the data sometimes lists multiple edges between the same nodes, but with different costs or capacities. In forming the graph, you need to implement your reduction from the previous question and form a `DiGraph` instance, because the `min_cost_flow` function cannot handle multi-edges, even though the package offers `MultiDiGraph` objects.

In [1]:
import networkx as nx

def create_graph(infile):
    file = open(infile).read().split('\n')
    countNodes = int([i.split(' ') for i in file if i!='' and i[0]=='p'][0][2])
    node = {i.split(' ')[1]:i.split(' ')[2] for i in file if i!='' and i[0]=='n'}
    arc = [i.split(' ') for i in file if i!='' and i[0]=='a']
    DG = nx.DiGraph()
    list = []
    for i in arc:
        if((i[2],i[1]) not in list):
            tmp = (i[2],i[1])
            list.append(tmp)
            DG.add_edge(i[2],i[1])
            DG.edge[i[2]][i[1]]['capacity'] = int(i[4])
            DG.edge[i[2]][i[1]]['weight'] = int(i[5])
        else:
            countNodes += 1
            countNodes = str(countNodes)
            DG.add_edge(i[2],countNodes)
            DG.add_edge(countNodes,i[1])
            DG.edge[i[2]][countNodes]['capacity'] = int(i[4])
            DG.edge[countNodes][i[1]]['capacity'] = int(i[4])
            DG.edge[i[2]][countNodes]['weight'] = int(i[5])
            DG.edge[countNodes][i[1]]['weight'] = 0
            countNodes = int(countNodes)
    for i in DG.nodes():
        if i in node:
            DG.node[i]['demand'] = int(node[i])
        else:
            DG.node[i]['demand'] = 0
    return DG

The following will check that your code outputs the expected min cost flow values on several test instances.

In [2]:
G_40 = create_graph('gte_bad.40')
G_6830 = create_graph('gte_bad.6830')
G_176280 = create_graph('gte_bad.176280')

print "Correct value for _40 instance:", nx.min_cost_flow_cost(G_40) == 52099553858
print "Correct value for _6830 instance:", nx.min_cost_flow_cost(G_6830) == 299390431788
print "Correct value for _176280 instance:", nx.min_cost_flow_cost(G_176280) == 510585093810

Correct value for _40 instance: True
Correct value for _6830 instance: True
Correct value for _176280 instance: True


## Linear Programming

Instead of using special-purpose min-cost flow solvers, you will now formulate the problems as linear programs and use general-purpose LP solvers to find the solutions.

### Question 3

Implement the following function to formulate the flow LP and return the optimal value (i.e., minimum cost over feasible flows).

In [3]:
import pulp

def lp_flow_value(G):
    f = {}
    for (s1,s2) in G.edges():
        f[(s1,s2)] = pulp.LpVariable("("+s1+","+s2+")", 0, G.edge[s1][s2]['capacity'], pulp.LpInteger)
    prob = pulp.LpProblem("The Min-Cost Flow Problem", pulp.LpMinimize)
    prob += pulp.lpSum([G.edge[i][j]['weight']*f[(i,j)] for (i,j) in G.edges()]), "Total Cost of Flow"
    for i in G.nodes():
        prob += sum(f[(s1,s2)] for (s1,s2) in G.edges() if s2==i)-sum(f[(s1,s2)] for (s1,s2) in G.edges() if s1==i) == G.node[i]['demand']           
    prob.solve()
    return pulp.value(prob.objective)

The following will check that the LP finds the same optimal values as previously.

In [4]:
print "Correct value for _40 instance:", lp_flow_value(G_40) == 52099553858
print "Correct value for _6830 instance:", lp_flow_value(G_6830) == 299390431788
print "Correct value for _176280 instance:", lp_flow_value(G_176280) == 510585093810

Correct value for _40 instance: True
Correct value for _6830 instance: True
Correct value for _176280 instance: True
